In [3]:
from weasyprint import HTML

def generate_html(order):
    items_html = "".join([
        f"<p>{item['quantity']} x {item['name']}<br>Instructions: {item.get('instructions', '')}</p>"
        for item in order['items']
    ])
    html_content = f"""
    <html>
    <body style="font-family: monospace; width: 80mm;">
        <h2 style="text-align: center;">Restaurant Name</h2>
        <p>Date: {order['date']}</p>
        <p>Order #: {order['order_number']}</p>
        <p>Table #: {order['table_number']}</p>
        <hr />
        {items_html}
        <hr />
        <p>Subtotal: ${order['subtotal']:.2f}</p>
        <p>Tax: ${order['tax']:.2f}</p>
        <p>Total: ${order['total']:.2f}</p>
        <hr />
        <p style="text-align: center;">Thank you for dining with us!</p>
    </body>
    </html>
    """
    return html_content


In [4]:
generate_html({
    "date": "2021-01-01",
    "order_number": 1,
    "table_number": 1,
    "items": [
        {"name": "Pizza", "quantity": 2, "instructions": "No cheese"},
        {"name": "Beer", "quantity": 1},
    ],
    "subtotal": 20.00,
    "tax": 1.60,
    "total": 21.60,
})

'\n    <html>\n    <body style="font-family: monospace; width: 80mm;">\n        <h2 style="text-align: center;">Restaurant Name</h2>\n        <p>Date: 2021-01-01</p>\n        <p>Order #: 1</p>\n        <p>Table #: 1</p>\n        <hr />\n        <p>2 x Pizza<br>Instructions: No cheese</p><p>1 x Beer<br>Instructions: </p>\n        <hr />\n        <p>Subtotal: $20.00</p>\n        <p>Tax: $1.60</p>\n        <p>Total: $21.60</p>\n        <hr />\n        <p style="text-align: center;">Thank you for dining with us!</p>\n    </body>\n    </html>\n    '

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 KB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 KB 6.4 MB/s eta 0:00:00
Using legacy 'setup.py install' for python-escpos, since package 'wheel' is not installed.
  Running setup.py install for python-escpos ... done
Note: you may need to restart the kernel to use updated packages.


In [14]:
from escpos.printer import Network

# Replace '192.168.1.100' with your printer's IP address
printer_ip = "10.0.0.138"

# Initialize the printer
printer = Network(printer_ip)

# Print a simple receipt
printer.text("\d0")
printer.cut()

# Close the connection
printer.close()


In [7]:
import StarTSPImage
from PIL import Image, ImageDraw
import socket
import time

# Create an image
image = Image.new('RGB', (500, 200), color='White')
draw = ImageDraw.Draw(image)
draw.ellipse((0, 0, 500, 500), fill='Black')
draw.ellipse((10, 10, 490, 490), fill='White')

# Convert the image to raster format
raster = StarTSPImage.imageToRaster(image, cut=True)

# Define the printer IP address and port
printer_ip = "10.0.0.138"  # Replace with your printer's IP address
printer_port = 9100  # Default port for most network printers

def send_raw_printer_data(ip, port, data):
    try:
        # Create a socket connection to the printer
        with socket.create_connection((ip, port), timeout=10) as sock:
            # Send raw data in smaller chunks if necessary
            chunk_size = 4096  # Adjust the chunk size if needed
            for i in range(0, len(data), chunk_size):
                sock.sendall(data[i:i + chunk_size])
                time.sleep(0.1)  # Add a small delay to allow the printer to process data
        print("Data sent to printer successfully.")
    except Exception as e:
        print(f"Failed to send data to printer: {e}")

# Send the raster data to the printer
send_raw_printer_data(printer_ip, printer_port, raster)


Data sent to printer successfully.


In [1]:
import imgkit
from io import BytesIO
from PIL import Image, ImageDraw
import datetime
import tempfile, os
import StarTSPImage

receipt_data = {
    'orderItems': [
        {
            'item_name': 'Cheeseburger',
            'price': 8.99,
            'quantity': 1,
            'configurations': [
                {'label': 'Cooking Preference', 'value': 'Medium'},
                {'label': 'Add-ons', 'value': 'Bacon'}
            ]
        },
        {
            'item_name': 'French Fries',
            'price': 2.99,
            'quantity': 1,
            'configurations': []
        }
    ],
    'price': 11.98
}

tz = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
order_date = datetime.datetime.now(tz).strftime('%m/%d/%Y %I:%M:%S %p')
html_content = f"""
<html>
<head>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Roboto:ital,wght@0,100;0,300;0,400;0,500;0,700;0,900;1,100;1,300;1,400;1,500;1,700;1,900&display=swap');
        body {{
            width: {80*8}mm;
            margin: 0;
            font-family: 'Roboto', sans-serif;
        }}
        p, span {{
            margin: {2*8}px 0;
            font-size: {32*8}px;
        }}
        .center {{
            text-align: center;
        }}
        .item {{
            display: flex;
            justify-content: space-between;
        }}
        .item-details {{
            padding-left: {6.7625*8}mm;
        }}
        hr {{
            border: none;
            border-top: {1*8}px solid #000;
        }}
    </style>
</head>
<body>
    <div class="center">
        <p>Shags Crab and Seafood</p>
        <p>1045 South Broadway</p>
        <p>Pennsville, NJ 08079</p>
    </div>
    <hr />
    <div class="center">
        <p>Order #{1}, {order_date}</p>
    </div>
    <hr />
"""
for item in receipt_data['orderItems']:
    html_content += f"""
    <div class="item">
        <span>{item['quantity']} × {item['item_name']}</span>
        <span>${item['price']:.2f}</span>
    </div>
    """
    for config in item['configurations']:
        html_content += f"""
        <div class="item-details">
            <p>{config['label']}: {config['value']}</p>
        </div>
        """
html_content += f"""
    <hr />
    <div class="item">
        <span>Total:</span>
        <span>${receipt_data['price']:.2f}</span>
    </div>
    <div class="item">
        <span>Total:</span>
        <span>${receipt_data['price']:.2f}</span>
    </div>
    <div class="item">
        <span>Total:</span>
        <span>${receipt_data['price']:.2f}</span>
    </div>
    <hr />
</body>
</html>
"""
options = {
    'width': 80,
}
# Generate image and save it to a temporary file
with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
    tmpfile_path = tmpfile.name
    imgkit.from_string(html_content, tmpfile_path, options={"format": "png", "width": 80})
# Open the temporary file with PIL for further manipulation
with open(tmpfile_path, "rb") as f:
    img = Image.open(BytesIO(f.read()))
# Save the manipulated image to a file
img.save("receipt_image_with_watermark.png")
# # Clean up the temporary file
# os.remove(tmpfile_path)

# raster = StarTSPImage.imageToRaster(img, cut=True)


# # Generate image and save it to a temporary file
# with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
#     tmpfile_path = tmpfile.name
#     imgkit.from_string(html_content, tmpfile_path, options={"format": "png"})

# # Open the temporary file with PIL for further manipulation
# with open(tmpfile_path, "rb") as f:
#     img = Image.open(BytesIO(f.read()))

# # Save the manipulated image to a file
# img.save("receipt_image_with_watermark.png")

# Clean up the temporary file
# os.remove(tmpfile_path)


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


In [151]:
from escpos import printer
p = printer.Network("10.0.0.253", 9100)
img = Image.open("receipt_image_with_watermark.png").convert("1").resize((312, 311))
width, height = img.size
pixels = img.load()
data = bytearray()
for y in range(height):
    for x in range(0, width, 8):
        byte = 0
        for bit in range(8):
            byte <<= 1
            if x + bit < width and pixels[x + bit, y] == 0:
                byte |= 1
        data.append(byte)
x_bytes = width // 8
y_dots = height
m = 0
xL = x_bytes & 0xFF
xH = (x_bytes >> 8) & 0xFF
yL = y_dots & 0xFF
yH = (y_dots >> 8) & 0xFF
p._raw(bytes([0x1D, 0x76, 0x30, m, xL, xH, yL, yH]))
p._raw(data)
# p._raw(raster)
p.image
# # Cut paper
# p.cut()
# p.close()

In [2]:
open_cash_drawer_command = b'\x1b\x70\x00\x19\xfa'
PRINTER_IP = "10.0.0.254"
PRINTER_PORT = 9100

def open_cash_drawer():
    try:
        # Create a socket connection to the printer
        printer_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        printer_socket.connect((PRINTER_IP, PRINTER_PORT))
        
        # Send the command to open the cash drawer
        printer_socket.sendall(open_cash_drawer_command)
        
        # Close the connection
        printer_socket.close()
        print("Cash drawer opened successfully.")
    except Exception as e:
        print(f"Error: {e}")

In [8]:
from escpos import printer
p = printer.Network(PRINTER_IP, PRINTER_PORT)
drawer_command = b'\x07'
p._raw(drawer_command)
p.close()

In [10]:
import socket

# Replace with the IP address of your printer
PRINTER_IP = "10.0.0.254"
PRINTER_PORT = 9100

# ESC/POS command to open the cash drawer (ASCII BEL <07h>)
drawer_command = b'\x07'

def open_cash_drawer():
    try:
        # Create a socket connection to the printer
        printer_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        printer_socket.connect((PRINTER_IP, PRINTER_PORT))
        
        # Send the command to open the cash drawer
        printer_socket.sendall(drawer_command)
        
        # Close the connection
        printer_socket.close()
        print("Cash drawer opened successfully.")
    except Exception as e:
        print(f"Error: {e}")

open_cash_drawer()


Cash drawer opened successfully.
